In [78]:
from sklearn.impute import SimpleImputer
import joblib
import pandas as pd
import numpy as np

Globas variables

In [79]:
checkpoint = '3'
team = 'jojos'
dataset_path = '../data/data_validation3.csv'

Load model, features, scaler and pca

In [80]:
# Load model
loaded_model = joblib.load('../models/stack_model.joblib')

# Load feature list
feature_list = joblib.load('../models/list_of_features.joblib')

# Load scaler
scaler = joblib.load('../models/scaler.joblib')

# Load pca
pca = joblib.load('../models/pca.joblib')

In [81]:
df = pd.read_csv(dataset_path)

# Work with non-numeric features
X_objects_features = df.select_dtypes(include=['object'])

X_encoded = pd.get_dummies(X_objects_features)

df = pd.concat([df.select_dtypes(exclude='object'), X_encoded], axis=1)

# Create missing features
missing_columns = list(set(feature_list) - set(df.columns))
zeros_array = np.zeros((df.shape[0], len(missing_columns)))
df_missing = pd.DataFrame(zeros_array, columns=missing_columns)
df =pd.concat([df,  df_missing], axis=1)

df = df[feature_list] # Only use feature_columns already worked

In [82]:
# Crear un objeto SimpleImputer
imputer = SimpleImputer(strategy='mean')  # Puedes cambiar 'mean' por 'median', 'most_frequent', o cualquier valor constante

# Aplicar la imputación a todas las columnas
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Scaler
df = scaler.transform(df)

# PCA
X = pca.transform(df)

### Predict

In [83]:
results = loaded_model.predict_proba(X)
print('len', len(loaded_model.predict(X)))

print(loaded_model.predict(X), results[:,1])

len 100
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [0.11628496 0.08543013 0.08135677 0.10144628 0.08681207 0.13589903
 0.07851452 0.14177544 0.12735921 0.10217584 0.12001615 0.08777128
 0.11824858 0.08818558 0.09982562 0.09002923 0.11832948 0.08657792
 0.09197004 0.10069366 0.09062064 0.08978616 0.09174818 0.12277636
 0.08450163 0.0801823  0.10342521 0.080668   0.0986873  0.11239793
 0.12349816 0.08515238 0.1159308  0.09647849 0.11567986 0.12363545
 0.10000259 0.10980886 0.07712408 0.22219797 0.1101055  0.09737491
 0.12803866 0.09705728 0.09305326 0.11791059 0.18218589 0.1091636
 0.09044404 0.08768975 0.0896582  0.13394765 0.08228093 0.09907189
 0.11653375 0.08858359 0.1290477  0.08443513 0.09100593 0.10543146
 0.09043659 0.13272848 0.31711814 0.10693151 0.13960749 0.10215011
 0.08638391 0.08169304 0.07356435 0.08395629 0.10778

### Save predictions

In [84]:
df_results = pd.DataFrame(results[:,1], columns=['predicciones'])
display(df_results.head())
df_results.to_parquet(f'../results/checkpoint{checkpoint}_{team}.parquet')

,predicciones
0,0.116285
1,0.085430
2,0.081357
3,0.101446
4,0.086812
